#**Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

# **Pre Processing Data**

In [ ]:
#loading the dataset
forestFire = pd.read_csv('/content/drive/MyDrive/forestfires.csv')
forestFire.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,fire area
0,7,5,mar,fri,80-90,20-30,<500,<10,<15,>50,>5,0,0
1,7,4,oct,tue,>90,>30,>500,<10,>15,<50,<5,0,0
2,7,4,oct,sat,>90,>30,>500,<10,<15,<50,<5,0,0
3,8,6,mar,fri,>90,>30,<500,<10,<15,>50,<5,1,0
4,8,6,mar,sun,80-90,>30,<500,>10,<15,>50,<5,0,0


In [ ]:
forestFire.tail()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,fire area
512,4,3,aug,sun,80-90,>30,>500,<10,>15,<50,<5,0,1
513,2,4,aug,sun,80-90,>30,>500,<10,>15,>50,>5,0,1
514,7,4,aug,sun,80-90,>30,>500,>10,>15,>50,>5,0,1
515,1,4,aug,sat,>90,>30,>500,<10,>15,<50,<5,0,0
516,6,3,nov,tue,<80,<20,<500,<10,<15,<50,<5,0,0


In [ ]:
forestFire.shape  #observing the size of the dataset

(517, 13)

In [ ]:
forestFire.isnull().sum()   #checking for missing data

X            0
Y            0
month        0
day          0
FFMC         0
DMC          0
DC           0
ISI          0
temp         0
RH           0
wind         0
rain         0
fire area    0
dtype: int64

In [ ]:
#deleting unwanted columns
del forestFire['X']
del forestFire['Y']


In [ ]:
forestFire.head() #checking if the columns are deleted or not

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,fire area
0,mar,fri,80-90,20-30,<500,<10,<15,>50,>5,0,0
1,oct,tue,>90,>30,>500,<10,>15,<50,<5,0,0
2,oct,sat,>90,>30,>500,<10,<15,<50,<5,0,0
3,mar,fri,>90,>30,<500,<10,<15,>50,<5,1,0
4,mar,sun,80-90,>30,<500,>10,<15,>50,<5,0,0


In [ ]:
forestFire.columns # checking the columns to work with

Index(['month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind',
       'rain', 'fire area'],
      dtype='object')

In [ ]:
forestFire.info() # checking the data type of the values in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   month      517 non-null    object
 1   day        517 non-null    object
 2   FFMC       517 non-null    object
 3   DMC        517 non-null    object
 4   DC         517 non-null    object
 5   ISI        517 non-null    object
 6   temp       517 non-null    object
 7   RH         517 non-null    object
 8   wind       517 non-null    object
 9   rain       517 non-null    int64 
 10  fire area  517 non-null    int64 
dtypes: int64(2), object(9)
memory usage: 44.6+ KB


# **Graph creation**

In [ ]:
!pip install pgmpy #installing pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.6 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import BayesianNetwork    #importing bayesian network model
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator # importing estimators

In [ ]:
#creating the bayesian network for forest fire detection
ForestfirePredictionModel = BayesianNetwork([('temp','FFMC'),('temp','DMC'),('temp','DC'), ('RH','FFMC'),('RH','DMC'),('rain','DC'), ('rain','FFMC'),('rain','DMC'),('wind','FFMC'),('wind','ISI'),('FFMC','ISI'),('DMC','fire area'),('DC','fire area'),('ISI','fire area')])

In [ ]:
#Using MaximumLikelihoodEstimator to get more robust parameters, The objective of Maximum Likelihood Estimation is to find the set of parameters (theta) that maximize the likelihood function
ForestfirePredictionModel.fit(forestFire,estimator=MaximumLikelihoodEstimator)

In [ ]:
# for our dataset fire area represents if forest fire occured or not, this line of code prints conditional probabilities of 'fire area' parameter
print(ForestfirePredictionModel.get_cpds('fire area'))

+--------------+------------+------------+-----+----------+----------+--------------------+
| DC           | DC(<500)   | DC(<500)   | ... | DC(>500) | DC(>500) | DC(>500)           |
+--------------+------------+------------+-----+----------+----------+--------------------+
| DMC          | DMC(20-30) | DMC(20-30) | ... | DMC(<20) | DMC(>30) | DMC(>30)           |
+--------------+------------+------------+-----+----------+----------+--------------------+
| ISI          | ISI(<10)   | ISI(>10)   | ... | ISI(>10) | ISI(<10) | ISI(>10)           |
+--------------+------------+------------+-----+----------+----------+--------------------+
| fire area(0) | 0.5        | 0.5        | ... | 0.5      | 0.484    | 0.3865546218487395 |
+--------------+------------+------------+-----+----------+----------+--------------------+
| fire area(1) | 0.5        | 0.5        | ... | 0.5      | 0.516    | 0.6134453781512605 |
+--------------+------------+------------+-----+----------+----------+----------

In [ ]:
#this line of code prints probabilities of 'temp' parameter
print(ForestfirePredictionModel.get_cpds('temp'))

+-----------+----------+
| temp(<15) | 0.218569 |
+-----------+----------+
| temp(>15) | 0.781431 |
+-----------+----------+


# **Inferencing**

In [ ]:
#Using Variable elimination method to infer the probabilites
from pgmpy.inference import VariableElimination
forestFire_infer = VariableElimination(ForestfirePredictionModel)

# **Inferencing using the user inputs**

In [ ]:
#this block of code prompts the user to input the categorical values and infers the probability of 'fire area' parameter

print('For temp Enter { <15 or >15 }')
print('For RH Enter { <50 or >50}')
print('For wind Enter {<5 or >5 }')
print('For rain Enter { 0:no or 1:yes }')
print('For FFMC Enter { <80 or 80-90 or >90 }')
print('For DMC Enter { <20 or 20-30 or >30 }')
print('For DC Enter { <500 or >500 }')
print('For ISI Enter { <10 or >10 }')

q = forestFire_infer.query(variables=['fire area'], evidence={
    'temp':str(input('Enter Temperature: ')),
    'RH':str(input('Enter RH: ')),
     'wind':str(input('Enter wind: ')),
    'rain':int(input('Enter rain: ')),
    'FFMC':str(input('Enter FFMC: ')),
    'DMC':str(input('Enter DMC: ')),
    'DC':str(input('Enter DC: ')),
    'ISI':str(input('Enter ISI: ')),
    })

print(q)

# 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'fire area'
# 80-90 	20-30 	<500 	<10 	<15 	>50 	>5 	0 	0

For temp Enter { <15 or >15 }
For RH Enter { <50 or >50}
For wind Enter {<5 or >5 }
For rain Enter { 0:no or 1:yes }
For FFMC Enter { <80 or 80-90 or >90 }
For DMC Enter { <20 or 20-30 or >30 }
For DC Enter { <500 or >500 }
For ISI Enter { <10 or >10 }
Enter Temperature: >15
Enter RH: >50
Enter wind: >5
Enter rain: 0
Enter FFMC: <80
Enter DMC: 20-30
Enter DC: >500
Enter ISI: <10
+--------------+------------------+
| fire area    |   phi(fire area) |
+==============+==================+
| fire area(0) |           0.5000 |
+--------------+------------------+
| fire area(1) |           0.5000 |
+--------------+------------------+
